In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
import gc
from skimage.metrics import peak_signal_noise_ratio, structural_similarity, mean_squared_error
from scipy.stats import pearsonr

In [3]:
def crop_and_straighten(img, threshold=10, visualize=False):
    if len(img.shape) == 2 or img.shape[2] == 1:
        gray = img if len(img.shape) == 2 else img[:, :, 0]
    else:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Normalize to 8-bit for contour processing
    gray_8bit = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX)
    gray_8bit = gray_8bit.astype(np.uint8)

    # Create binary mask for valid region
    _, mask = cv2.threshold(gray_8bit, threshold, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        raise ValueError("No valid region found!")

    # Largest contour
    cnt = max(contours, key=cv2.contourArea)
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.intp(box)
    # print(box)

    width, height = int(rect[1][0]), int(rect[1][1])
    dst_pts = np.array([[0, height-1],
                        [0, 0],
                        [width-1, 0],
                        [width-1, height-1]], dtype="float32")
    src_pts = box.astype("float32")

    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(img, M, (width, height))

    if visualize:
        plt.imshow(warped, cmap='gray' if len(warped.shape) == 2 else None)
        plt.title("Straightened & Cropped Image")
        plt.axis("off")
        plt.show()

    return warped


def compute_quality_metrics(ref_img, test_img, PSNR=True):
    assert ref_img.shape == test_img.shape, "Images must have the same shape"
    assert ref_img.ndim == 3, "Images must be H x W x C"

    psnr_list = []
    cc_list = []
    ssim_list = []
    rmse_list = []

    for i in range(ref_img.shape[2]):
        ref_band = ref_img[:, :, i].astype(np.float64)
        test_band = test_img[:, :, i].astype(np.float64)

        # Use fixed data range for uint16 images
        data_range = 65535.0

        if PSNR:
          psnr_val = peak_signal_noise_ratio(ref_band, test_band, data_range=data_range)
          psnr_list.append(psnr_val)


        # cc_val, _ = pearsonr(ref_band.flatten(), test_band.flatten())
        # ssim_val = structural_similarity(ref_band, test_band, data_range=data_range)
        else:
          rmse_val = np.sqrt(mean_squared_error(ref_band, test_band))
          mean_ref = np.mean(ref_band)
          rel_rmse = rmse_val / mean_ref if mean_ref != 0 else 0
          rmse_list.append(rel_rmse)


        # cc_list.append(cc_val)
        # ssim_list.append(ssim_val)

        if PSNR:
            return {
              "PSNR": np.mean(psnr_list)
            }
        else:
            return {
                "RMSE": np.mean(rmse_list)
            }

def calculate_ergas(original_ms, pansharpened_ms, ratio):
    if original_ms.shape != pansharpened_ms.shape:
        raise ValueError("Images must be the same shape. Resample the original MS image first.")

    N = original_ms.shape[2]
    ergas_sum = 0

    for i in range(N):
        ref_band = original_ms[:, :, i].astype(np.float64)
        pan_band = pansharpened_ms[:, :, i].astype(np.float64)

        rmse = np.sqrt(mean_squared_error(ref_band, pan_band))
        mean_val = np.mean(ref_band)

        ergas_sum += (rmse / mean_val) ** 2

    ergas = 100 * (ratio) * np.sqrt(ergas_sum / N)
    return ergas


def compute_sam(img1, img2, eps=1e-8):
    # Flatten spatial dimensions
    X = img1.reshape(-1, img1.shape[-1]).astype(np.float32)
    Y = img2.reshape(-1, img2.shape[-1]).astype(np.float32)

    # Compute dot product and norms
    dot_product = np.sum(X * Y, axis=1)
    norm_x = np.linalg.norm(X, axis=1)
    norm_y = np.linalg.norm(Y, axis=1)

    # Avoid division by zero
    denom = np.clip(norm_x * norm_y, eps, None)

    # Compute angle (in radians)
    sam = np.arccos(np.clip(dot_product / denom, -1.0, 1.0))
    sam_map = sam.reshape(img1.shape[0], img1.shape[1])

    mean_sam = np.mean(sam)

    sam_map_safe = np.clip(sam_map, 1e-6, None)

    # Normalize to [0, 1]
    sam_map_norm = sam_map_safe / np.max(sam_map_safe)

    # Apply log transform
    sam_log = np.log1p(sam_map_norm)  # log(1 + x)

    # Normalize again and scale to 0-255
    sam_log_norm = (sam_log / np.max(sam_log)) * 255
    return sam_log_norm.astype(np.uint8) , mean_sam

def visualize_sam_map(sam_map, cmap='inferno'):
    plt.figure(figsize=(8, 6))
    plt.imshow(np.degrees(sam_map), cmap=cmap)
    plt.colorbar(label='SAM (degrees)')
    plt.title("Spectral Angle Mapper (SAM) Map")
    plt.axis('off')
    plt.tight_layout()
    plt.show()


In [4]:

# Read the TIFF image
image_b = tiff.imread('/content/LC08_L1TP_B2.tiff')
image_g = tiff.imread('/content/LC08_L1TP_B3.tiff')
image_r = tiff.imread('/content/LC08_L1TP_B4.tiff')
image_ir = tiff.imread('/content/LC08_L1TP_B5.tiff')
image_pan = tiff.imread('/content/LC08_L1TP_B8.tiff')

# Check if image is loaded successfully
if image_b is None:
    print("Failed to load image.")
else:
    print("Image shape:", image_b.shape)
    # Display the image


Image shape: (7951, 7821)


In [5]:
image_b = crop_and_straighten(image_b,visualize=False);
image_g = crop_and_straighten(image_g,visualize=False);
image_r = crop_and_straighten(image_r,visualize=False);
image_ir = crop_and_straighten(image_ir,visualize=False);
image_pan = crop_and_straighten(image_pan,visualize=False);

In [6]:
print(image_b.shape)
print(image_g.shape)
print(image_r.shape)
print(image_ir.shape)
print(image_pan.shape)

(6552, 6350)
(6552, 6351)
(6552, 6350)
(6552, 6351)
(13105, 12715)


In [7]:
image_b =  cv2.resize(image_b, (6348, 6550), interpolation=cv2.INTER_LINEAR)
image_r =  cv2.resize(image_r, (6348, 6550), interpolation=cv2.INTER_LINEAR)
image_g =  cv2.resize(image_g, (6348, 6550), interpolation=cv2.INTER_LINEAR)
image_ir =  cv2.resize(image_ir, (6348, 6550), interpolation=cv2.INTER_LINEAR)

In [8]:
ms_image = np.stack([image_b,image_g,image_r,image_ir],axis=-1)

In [9]:
ms_image.shape

(6550, 6348, 4)

In [10]:
tiff.imwrite('Blue band.tiff',image_b)
tiff.imwrite('Green band.tiff',image_g)
tiff.imwrite('Red band.tiff',image_r)
tiff.imwrite('Infra Red band.tiff',image_ir)

del image_b
del image_r
del image_g
del image_ir
gc.collect()

615

#### Upsample MS images to size of PAN

In [11]:
(pan_height,pan_width) = image_pan.shape[:2]

In [12]:
ms_image_resize = cv2.resize(ms_image, (pan_width, pan_height), interpolation=cv2.INTER_LINEAR)

In [13]:
pan_filtered =  cv2.GaussianBlur(image_pan, (5, 5), sigmaX=1.0)

In [14]:
pan_hf = image_pan - pan_filtered

In [15]:
del pan_filtered
gc.collect()

0

In [16]:
pan_sharpened_image = ms_image_resize + pan_hf[:, :, np.newaxis]

In [17]:
pan_sharpened_image.shape

(13105, 12715, 4)

In [18]:
del ms_image_resize
gc.collect()

0

In [19]:
# Resize PAN-sharpened image to MS resolution
pan_sharpened_down = cv2.resize(pan_sharpened_image, (ms_image.shape[1], ms_image.shape[0]),interpolation=cv2.INTER_AREA)


In [20]:
pan_sharpened_down.shape

(6550, 6348, 4)

In [21]:
tiff.imwrite('Pansharpened_image.tiff',pan_sharpened_image[:,:,:3])
tiff.imwrite('Multispectral_rgb_image.tiff',ms_image[:,:,:3])
tiff.imwrite('PAN_HF_image.tiff',pan_hf)

In [22]:
del pan_sharpened_image
del pan_hf
gc.collect()

0

In [24]:
sam_image, mean_sam = compute_sam(ms_image,pan_sharpened_down)
psnr = compute_quality_metrics(ms_image,pan_sharpened_down)
ergas = calculate_ergas(ms_image,pan_sharpened_down,4)
rmse = compute_quality_metrics(ms_image,pan_sharpened_down,False)

In [25]:
print(psnr,rmse)

{'PSNR': np.float64(34.40016652619282)} {'RMSE': np.float64(0.12448642608601042)}


In [26]:
del ms_image
del pan_sharpened_down
gc.collect()

200

In [27]:
print("Mean SAM (rad):", mean_sam)
print("Mean SAM (deg):", np.degrees(mean_sam))
print("ERGAS:", ergas)

Mean SAM (rad): 0.010222357
Mean SAM (deg): 0.5856979
ERGAS: 50.987598612105934
